In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt;
import seaborn as sns; 
import numpy as np;
%matplotlib inline

In [ ]:
dirname =  '/kaggle/input/real-estate-price-prediction/'
filename = 'Real estate.csv'

data = pd.read_csv(os.path.join(dirname,filename))
print('Num columns:',data.shape[0],'\n'+'Num rows:',data.shape[1])
n=data.shape[0]
data.head(10)

In [ ]:
# Drop the 'No' column as it's redunant 

data.drop(['No'],axis=1,inplace=True)
data.head(10)

In [ ]:
# The only discrete column is X4 (and sort of X1) however, we only really need to discretise labels when there are no clear 'levels' to the data
# However, here X4 might naturally have levels i.e. higher number = better. 
# Can check this by plotting it against the response variable

# Draw line of best fit for said data as well
x = np.concatenate([np.array([1]*n).reshape(n,1),np.array(data['X4 number of convenience stores']).reshape(n,1)],axis=1).reshape(n,2)
y = np.array(data['Y house price of unit area']).reshape(n,1)
B = np.linalg.inv(x.T@x) @ x.T @ y

plt.figure(figsize=(15,10))
plt.scatter(data['X4 number of convenience stores'],data['Y house price of unit area'])
plt.plot(data['X4 number of convenience stores'].values,B[0]+B[1]*data['X4 number of convenience stores'].values,color='orange',linewidth=2)
plt.xlabel('Number of conveniance stores')
plt.ylabel('Y')
plt.legend(['OLS Fit','Actual Data'])
print('Intercept='+str(B[0]),'Slope'+str(B[1]))

Can see that on average, higher numbers of convenience stores responds to a higher response variable so I will leave it as it is (i.e. continuous).

In [ ]:
# Observing the transaction data vs house price:

x = np.concatenate([np.array([1]*n).reshape(n,1),np.array(data['X1 transaction date']).reshape(n,1)],axis=1).reshape(n,2)
y = np.array(data['Y house price of unit area']).reshape(n,1)
B = np.linalg.inv(x.T@x) @ x.T @ y

plt.figure(figsize=(15,10))
plt.scatter(data['X1 transaction date'],data['Y house price of unit area'])
plt.plot(data['X1 transaction date'].values,B[0]+B[1]*data['X1 transaction date'].values,color='orange',linewidth=2)
plt.xlabel('Transaction Date')
plt.ylabel('Y')
plt.legend(['OLS Fit','Actual Data'])
print('Intercept='+str(B[0]),'Slope'+str(B[1]))

I will remove this feature as the transaction date variable as I believe it's a relatively uninformative variable when looking at it plotted against the responve variable.
Additionally, I would prefer as fewer features and as sparse model as possible for simplicity.


In [ ]:
data.drop(['X1 transaction date'],axis=1,inplace=True)
data.head(10)

In [ ]:
# Check for duplicated rows

assert len(data.duplicated()[data.duplicated()==True]) == 0

In [ ]:
# For a preprocessing step, I will scale the input features using a standard scaler - subtracts by the mean and scales by 1/s.d.

from sklearn.preprocessing import StandardScaler
import numpy as np

sc = StandardScaler()

X = sc.fit_transform(np.array(data.drop(['Y house price of unit area'],axis=1)))
y = np.array(data['Y house price of unit area'])

# Additionally, before fitting a regression model, I will add a column of ones such that an intercept can be fitted and the data doesn't have to pass through the origin 

ones = np.ones(len(X)).reshape(len(X),1)
X = np.concatenate([ones,X],axis=1).reshape(len(X),X.shape[1]+1)

print('Num Instances',X.shape[0],'Num Features',X.shape[1])

**Linear Models**

I will firstly explore the peformance of two linear models: a standard linear regression model $y = \beta_{0} + \beta_{1}X_{1} + ... + \beta_{N}X_{N}$ and *ridge regression* model in which we add a penalty term to the cost curve $J(\mathbf{\beta})$ such that if using RSS we obtain $J(\mathbf{\beta}) = RSS(\mathbf{\beta}) + \lambda \sum_{i=1}^{N}\beta_{i}$ for the new cost function minimise in training. We end up using a slightly modified version of least-squares to train and fit parameters.

Both models will be fit and then tested using 10-fold cross validation, with MSE averaged over the 10 folds for an aggregate score.

In [ ]:
# Standard linear model 
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

kf = KFold(10,shuffle=True)

scores = []
lr_coefs = []

for i,j in kf.split(X,y):
    reg = LinearRegression().fit(X[i],y[i])
    lr_coefs.append(reg.coef_)
    preds = reg.predict(X[j])
    mse = mean_squared_error(y[j],preds)
    scores.append(mse)
    
print("Average MSE:",np.mean(scores),'\n'+"Std MSE:",np.std(scores))
print("Max MSE:",max(scores),'\n'+"Min MSE:",min(scores))


In [ ]:
# Lasso regression model

ls_coefs = []

for i,j in kf.split(X,y):
    ls = Lasso().fit(X[i],y[i])
    ls_coefs.append(ls.coef_)
    preds = ls.predict(X[j])
    mse = mean_squared_error(y[j],preds)
    scores.append(mse)
    
print("Average MSE:",np.mean(scores),'\n'+"Std MSE:",np.std(scores))
print("Max MSE:",max(scores),'\n'+"Min MSE:",min(scores))

In [ ]:
# Inspecting differences in coefficients between the two methodologies 

s = "Standard regression average coefficients: B0= %s, B1= %s, B2= %s, B3= %s, B4= %s, B5= %s" % tuple(np.mean(np.array(lr_coefs),axis=0))
print(s)

ss = "Lasso regression average coefficients: B0= %s, B1= %s, B2= %s, B3= %s, B4= %s, B5= %s" % tuple(np.mean(np.array(ls_coefs),axis=0))
print(ss)

The real difference we observe in the 2 methodologies is a shrinking of the coefficient $B_{5}$ down to 0 in the lasso regression model verus a very small coefficient value for the standard regression model. 
This is the purpose of lasso regression, to rather than just shrink coefficients to small values but set them to 0 for greater interpretability and model simplicity. 

**Dimensionality Reduction**

I am curious to see how performance will be affted by reducing the dimensionality of the data. I will choose a simple method of reduction: PCA, where we perform orthogonal projections of the data onto the axes of a lower dimensional vector space. 

In [ ]:
from sklearn.decomposition import PCA 

# Firstly I will calculate only the first principle component 
pca = PCA(n_components=1)
pc_1 = pca.fit_transform(X)

kf = KFold(10)
scores= [] 
for i,j in kf.split(pc_1,y):
    reg = LinearRegression().fit(pc_1[i],y[i])
    preds = reg.predict(pc_1[j])
    mse = mean_squared_error(y[j],preds)
    scores.append(mse)
    
print("Average MSE:",np.mean(scores),'\n'+"Std MSE:",np.std(scores))

# For the first 2 components
pca = PCA(n_components=2)
pc_2 = pca.fit_transform(X)

kf = KFold(10)
scores= [] 
for i,j in kf.split(pc_1,y):
    reg = LinearRegression().fit(pc_2[i],y[i])
    preds = reg.predict(pc_2[j])
    mse = mean_squared_error(y[j],preds)
    scores.append(mse)
    
print("Average MSE:",np.mean(scores),'\n'+"Std MSE:",np.std(scores))

The results show that using 2 components is only marginally worse than using the normal dataset as we see an average MSE of 85.7 compared with 81.4 for standard regression. If we wanted a more highly interpretable model maybe we would stick with first-2 principal components, and if we cared more about prediction would keep the full set of features.

In [ ]:
# Plot the first principal component against the response values

plt.figure(figsize=(15,10))
plt.scatter(pc_1,y)

The data appears to be slightly nonlinear - could attempt to fit a nonlinear regression using only the first principal component.

In [ ]:
# Create new data set including a squared term for the first principal component and an intercept 

non_lin_data = np.concatenate([np.ones(len(pc_1)).reshape(len(pc_1),1),pc_1,pc_1**2],axis=1).reshape(len(pc_1),3)

# Can now simply fit a linear regression model again - the model is still linear in the weights/coefficients

Xtr, Xts, ytr, yts = train_test_split(non_lin_data,y,test_size=0.25,shuffle=True)

reg = LinearRegression().fit(Xtr,ytr)
coefs = reg.coef_
preds = reg.predict(Xts)
mse = mean_squared_error(yts,preds)
print("MSE:",mse)

In [ ]:
line = coefs[0]+coefs[1]*pc_1 + coefs[2]*pc_1**2
unsorted_data = np.concatenate([pc_1,line],axis=1).reshape(len(line),2)
sorted_data=unsorted_data[unsorted_data[:,0].argsort()]

plt.figure(figsize=(15,10))
plt.scatter(pc_1, y)
plt.plot(sorted_data[:,0], sorted_data[:,1] ,color='orange',linewidth=3)

Ultimately, the best model I produced and tested was the nonlinear regression using only the first principal component. Only using the first PC allowed me to observe the structure of the data and the relationship to the response variable therefore allowing me to best choose the best exponent to raise the data to for a nonlinear fit. 